<a href="https://colab.research.google.com/github/castudil/som-lstm/blob/main/algoritmo_perturbacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline

# Funcion de Perturbacion



In [149]:
iris = sns.load_dataset("iris")


In [178]:
def perturbation(df, col=0, mu=0, sigma=0.1):

  """
    Funcion que perturba una columna de un dataframe, con una cantidad 
    normalmente distribuida con media 0 y desviación típica 0,1 por defecto.

    Parameters
        -----------
        df : DataFrame 
          DataFrame de los datos para perturbar
        col : int
          columna del DataFrame que sera perturbada
        mu: float
          mean
        sigma: float
          desviacion estandar
  """
  df = df.copy() # copia el dataframe para evitar que se modifique el dataframe original

  cardinality = dataset.shape[0] # numero de instancias del dataset

  # Toma muestras aleatorias de una distribución normal
  perturbacion = pd.Series(np.random.normal(loc=mu, scale=sigma, size=cardinality))

  columns_name  = df.columns # nombres de las columnas

  # Suma las muestras aleatorias a la columna seleccionada y pasa esta suma al dataset copiado.
  df[columns_name[col]] = df[columns_name[col]] + perturbacion  

  return df

## Test de perturbaciones 

In [179]:
p1 = perturbation(df=iris, col=0)

print(p1.head())
print(iris.head())



   sepal_length  sepal_width  petal_length  petal_width species
0      4.917984          3.5           1.4          0.2  setosa
1      4.915681          3.0           1.4          0.2  setosa
2      4.661711          3.2           1.3          0.2  setosa
3      4.566688          3.1           1.5          0.2  setosa
4      4.909786          3.6           1.4          0.2  setosa
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [177]:
p2 = perturbation(iris, col=3)

print(p2.head())
print(iris.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4     0.112949  setosa
1           4.9          3.0           1.4     0.256411  setosa
2           4.7          3.2           1.3     0.276237  setosa
3           4.6          3.1           1.5     0.153464  setosa
4           5.0          3.6           1.4     0.243559  setosa
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


# Modelos

## Cargar dataset

In [187]:
path = 'https://raw.githubusercontent.com/castudil/som-lstm/main/data/talca.csv' # use your path

df = pd.read_csv(path, index_col=None, header=0)
# Convert the date column to a datestamp type
df['date'] = pd.to_datetime(df['date'])
# Set the date column as the index of your DataFrame meat
df = df.set_index('date')

print(df.sample(5))

print(df.describe())


                       PM25
date                       
2017-02-09 22:00:00   7.227
2017-09-04 05:00:00  17.000
2018-01-09 04:00:00   3.000
2020-05-08 00:00:00  52.000
2019-02-03 02:00:00   5.000
               PM25
count  43847.000000
mean      16.653470
std       27.244984
min        0.000000
25%        3.000000
50%        8.000000
75%       19.000000
max      766.200000
